In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Lambda, BatchNormalization, Flatten
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

In [3]:
X_train = np.load('../X_train.npy')
X_valid = np.load('../X_valid.npy')
y_train = np.load('../y_train.npy')
y_valid = np.load('../y_valid.npy')
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(36000, 784) (36000, 10)
(6000, 784) (6000, 10)


In [11]:
mean_px = X_train.mean()
std_px = X_train.std()
def normalize(x):
    return (x - mean_px) / std_px

In [24]:
model = Sequential()
model.add(BatchNormalization(input_shape=(28,28,1), axis=-1))
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(BatchNormalization(axis=-1))
#model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu'))
#model.add(BatchNormalization(axis=-1))
#model.add(Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu'))
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [25]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_13 (Batc (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization_14 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 6272)              25088     
__________

In [27]:
params = {}
params['x'] = X_train.reshape(36000,28,28,1)
params['y'] = y_train
params['validation_data'] = (X_valid.reshape(6000,28,28,1), y_valid)

In [28]:
model.optimizer.lr = 0.001
params['epochs'] = 1
model.fit(**params)

Train on 36000 samples, validate on 6000 samples
Epoch 1/1
36000/36000 [==============================] - 74s - loss: 0.2039 - acc: 0.9410 - val_loss: 0.1046 - val_acc: 0.9650


In [29]:
model.save_weights('../digi_model/model_1.h5')

In [30]:
X_score = np.load('../X_score.npy').reshape(28000,28,28,1)

In [31]:
predict = model.predict_classes(X_score)

27904/28000 [============================>.] - ETA: 0s

In [38]:
predict = pd.DataFrame(predict, columns=['Label'])
predict = predict.reset_index()
predict.columns = ['ImageId','Label']
predict['ImageId'] = predict['ImageId'] + 1


In [40]:
predict.to_csv('../digi_model/submit_v1.csv', index=False)
